In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r'C:\Users\anaverageone\htlt_env\Advanced_NLP')
print(os.getcwd())

C:\Users\anaverageone\htlt_env\Advanced_NLP


In [2]:
# train_path = 'data/train_split.tsv'
# header_split_df = ['Copy_ID','Sent_ID','ID','FORM','LEMMA','UPOS','XPOS','FEATS','HEAD','DEPREL','DEPS','MISC','UP:PRED','UP:ARGHEADS']
train_path, test_path = 'data/train_split.tsv', 'data/test_split.tsv'
train_df, test_df = pd.read_csv(train_path, sep='\t', encoding='utf-8',quotechar='№'), pd.read_csv(test_path, sep='\t', encoding='utf-8',quotechar='№')

C:\Users\anaverageone\AppData\Local\Temp\ipykernel_13904\1232287190.py:4: ParserWarning: Falling back to the 'python' engine because ord(quotechar) > 127, meaning the quotechar is larger than one byte, and the 'c' engine does not support such quotechars; you can avoid this warning by specifying engine='python'.
  train_df, test_df = pd.read_csv(train_path, sep='\t', encoding='utf-8',quotechar='№'), pd.read_csv(test_path, sep='\t', encoding='utf-8',quotechar='№')
C:\Users\anaverageone\AppData\Local\Temp\ipykernel_13904\1232287190.py:4: ParserWarning: Falling back to the 'python' engine because ord(quotechar) > 127, meaning the quotechar is larger than one byte, and the 'c' engine does not support such quotechars; you can avoid this warning by specifying engine='python'.
  train_df, test_df = pd.read_csv(train_path, sep='\t', encoding='utf-8',quotechar='№'), pd.read_csv(test_path, sep='\t', encoding='utf-8',quotechar='№')


In [7]:
train_df, test_df = train_df.astype({'Sent_ID':'int'}), test_df.astype({'Sent_ID':'int'})
train_df, test_df = train_df.astype({'Copy_ID':'int'}), test_df.astype({'Copy_ID':'int'})
train_df, test_df = train_df.astype({'ID':'int'}), test_df.astype({'ID':'int'})
train_df, test_df = train_df.astype({'HEAD':'int'}), test_df.astype({'HEAD':'int'})

In [22]:
train = train_df[:100]
test = test_df[:50]

In [8]:
parent_dep_rel = test.loc[test['ID'] == 4, 'DEPREL'].values[0]
print(type(parent_dep_rel))

<class 'str'>


In [14]:
features_dict_list = []
copy_id_list = train['Copy_ID'].unique()
    
for num in copy_id_list:
    df_copy = train.loc[train['Copy_ID'] == num]
    max_wds_count = df_copy['ID'].max()
    pred_item = df_copy['ID'][df_copy['UP:PRED'].str.len() > 2]
    # pred_id = None
    if not pred_item.empty:
        pred_id = pred_item.values[0]
        pred_form = df_copy.loc[df_copy['ID'] == pred_id,'FORM'].values[0]
    else:
        pred_form = 0

    for i in range(max_wds_count):
        features_dict = {}
        path = []
        df_row = df_copy.iloc[i]
        row_head = df_row['HEAD']
        row_dep_rel = df_row['DEPREL']
        path.append(row_dep_rel)
        if row_head == 0:
            head_form = 0
        else:
            head_rows = df_copy.loc[df_copy['ID'] == row_head,'FORM']
            if not head_rows.empty:
                head_form = head_rows.values[0] 
            else:
                head_form = 0   
        features_dict['PRED_HEAD']= f"{pred_form}_{head_form}"
        
#         while row_head != 0:
#             parent_row = df_copy.loc[df_copy['ID'] == row_head,'HEAD']   
#             if not parent_row.empty:
#                 parent_head = parent_row.values[0]
#                 parent_dep_rel = df_copy.loc[df_copy['ID'] == parent_head, 'DEPREL'].values[0]
                
#             else:
#                 parent_dep_rel = 0
#             path.append(parent_dep_rel)   
#             # path = list(dict.fromkeys(path))
#         features_dict['PATH']= path

        features_dict_list.append(features_dict)

print(features_dict_list)

[{'PRED_HEAD': 'killed_0'}, {'PRED_HEAD': 'killed_Al'}, {'PRED_HEAD': 'killed_Al'}, {'PRED_HEAD': 'killed_Al'}, {'PRED_HEAD': 'killed_forces'}, {'PRED_HEAD': 'killed_killed'}, {'PRED_HEAD': 'killed_Al'}, {'PRED_HEAD': 'killed_killed'}, {'PRED_HEAD': 'killed_Shaikh'}, {'PRED_HEAD': 'killed_Shaikh'}, {'PRED_HEAD': 'killed_Shaikh'}, {'PRED_HEAD': 'killed_Shaikh'}, {'PRED_HEAD': 'killed_Shaikh'}, {'PRED_HEAD': 'killed_preacher'}, {'PRED_HEAD': 'killed_Shaikh'}, {'PRED_HEAD': 'killed_mosque'}, {'PRED_HEAD': 'killed_mosque'}, {'PRED_HEAD': 'killed_killed'}, {'PRED_HEAD': 'killed_town'}, {'PRED_HEAD': 'killed_town'}, {'PRED_HEAD': 'killed_mosque'}, {'PRED_HEAD': 'killed_Qaim'}, {'PRED_HEAD': 'killed_town'}, {'PRED_HEAD': 'killed_town'}, {'PRED_HEAD': 'killed_border'}, {'PRED_HEAD': 'killed_border'}, {'PRED_HEAD': 'killed_border'}, {'PRED_HEAD': 'killed_town'}, {'PRED_HEAD': 'killed_Al'}, {'PRED_HEAD': 'killing_causing'}, {'PRED_HEAD': 'killing_killing'}, {'PRED_HEAD': 'killing_causing'}, {'PR

In [49]:
features_dict_list = []
copy_id_list = train['Copy_ID'].unique()
    
for num in copy_id_list:

    df_copy = train.loc[train['Copy_ID'] == num]
    max_wds_count = df_copy['ID'].max()

    for i in range(max_wds_count):
        features_dict = {}
        path = []
        df_row = df_copy.iloc[i]
        row_head = df_row['HEAD']
        row_dep_rel = df_row['DEPREL']
        # row_dep_rel = ":".join(list(filter(lambda x: x.isalpha(), df_row['DEPS'].split(":"))
        path.append(row_dep_rel) 
        
        while (row_head != 0).any():
            parent_row = df_copy.loc[df_copy['ID'] == int(row_head),'HEAD']   
            if not parent_row.empty:
                parent_head = parent_row.values[0]
                parent_dep_rel_values = df_copy.loc[df_copy['ID'] == parent_head, 'DEPREL'].values
                if parent_dep_rel_values.size > 0:
                    parent_dep_rel = parent_dep_rel_values[0]
                    # parent_dep_rel = ":".join(list(filter(lambda x: x.isalpha(), parent_dep_rel_values[0].split(":"))
                    path.append(parent_dep_rel) 
                    row_head = df_copy.loc[df_copy['ID'] == parent_head,'HEAD']
                else:
                    row_head = pd.Series([0])
        if 'root' not in path:
            path.append('root')
        features_dict['PATH']= path
        print(i+1 ,features_dict)
        
        
        features_dict_list.append(features_dict)
       


1 {'PATH': ['root']}
2 {'PATH': ['punct', 'root']}
3 {'PATH': ['flat', 'root']}
4 {'PATH': ['punct', 'root']}
5 {'PATH': ['amod', 'parataxis', 'root']}
6 {'PATH': ['nsubj', 'root']}
7 {'PATH': ['parataxis', 'root']}
8 {'PATH': ['obj', 'root']}
9 {'PATH': ['flat', 'parataxis', 'root']}
10 {'PATH': ['flat', 'parataxis', 'root']}
11 {'PATH': ['punct', 'parataxis', 'root']}
12 {'PATH': ['flat', 'parataxis', 'root']}
13 {'PATH': ['punct', 'parataxis', 'root']}
14 {'PATH': ['det', 'obj', 'root']}
15 {'PATH': ['appos', 'parataxis', 'root']}
16 {'PATH': ['case', 'parataxis', 'root']}
17 {'PATH': ['det', 'parataxis', 'root']}
18 {'PATH': ['obl', 'root']}
19 {'PATH': ['case', 'obl', 'root']}
20 {'PATH': ['det', 'obl', 'root']}
21 {'PATH': ['nmod', 'parataxis', 'root']}
22 {'PATH': ['case', 'nmod', 'parataxis', 'root']}
23 {'PATH': ['nmod', 'obl', 'root']}
24 {'PATH': ['punct', 'obl', 'root']}
25 {'PATH': ['case', 'nmod', 'parataxis', 'root']}
26 {'PATH': ['det', 'nmod', 'parataxis', 'root']}
27 

In [45]:
features_dict_list = []
copy_id_list = train['Copy_ID'].unique()
    
for num in copy_id_list:
    df_copy = train.loc[train['Copy_ID'] == num]
    max_wds_count = df_copy['ID'].max()

    for i in range(max_wds_count):
        features_dict = {}
        path = []
        df_row = df_copy.iloc[i]
        row_head = df_row['HEAD']
        row_dep_rel = df_row['DEPS']
        row_deps = ":".join(list(filter(lambda x: x.isalpha(), row_dep_rel.split(":"))
        path.append(row_deps) 
        
        while (row_head != 0).any():
            parent_row = df_copy.loc[df_copy['ID'] == int(row_head),'HEAD']   
            if not parent_row.empty:
                parent_head = parent_row.values[0]
                parent_dep_rel_values = df_copy.loc[df_copy['ID'] == parent_head, 'DEPS'].values
                if parent_dep_rel_values.size > 0:
                    parent_dep_rel = parent_dep_rel_values[0]
                    parent_deps = ":".join(list(filter(lambda x: x.isalpha(), parent_dep_rel.split(":"))
                    path.append(parent_deps) 

                    row_head = df_copy.loc[df_copy['ID'] == parent_head,'HEAD']
                else:
                    row_head = pd.Series([0])
                    
        if 'root' not in path:
            path.append('root')
    
        features_dict['PATH']= path
        features_dict_list.append(features_dict)
       
print(features_dict_list)

SyntaxError: invalid syntax (196453977.py, line 15)

In [ ]:
#         if int(head_ix) == 0:
#             head_form = '0'
#         else:
#             head_form = df_copy.loc[(df_copy['ID'] == head_ix),'FORM'].squeeze()
                
#             # predicate of current phrase
#             up_pred = df_copy.loc[(df_copy['UP:PRED'] != '_'),'FORM'].squeeze()

#             # Feature - predicate & head combination
#             feature_dict['PRED_HEAD']= up_pred + '_' + head_form
            
#             # Feature - Path from constituent to root
#             while int(head_ix) != 0:
#                 # head_ix = df_copy.loc[df_copy['ID'] == ,'HEAD'].squeeze()
#                 head_ix = df_copy.loc[df_copy['ID'] == int(head_ix), 'HEAD']
#                 if int(head_ix) == 0:
#                     dep_rel = 'root'
#                 else:
#                     dep_rel = df_copy.loc[df_copy['ID'] == int(head_ix),'DEPREL'].squeeze()
#                 paths.append(dep_rel)
#                 paths = list(dict.fromkeys(paths))
            
#             features_dict_list.append(features_dict) 
        
#     return features_dict_list